In [1]:
# encoding=utf-8
"""
    Created on 10:38 2018/11/10 
    @author: Jindong Wang
"""


import os
os.getcwd()

'/Users/lizliao/Downloads/GitFYP_experiment/wip:tut:CNN-lstm-torch/pytorch'

In [2]:
path = '/Users/lizliao/Downloads/GitFYP_experiment/Dataset/UCI HAR Dataset/data/'
os. chdir(path)
os.getcwd()

'/Users/lizliao/Downloads/GitFYP_experiment/Dataset/UCI HAR Dataset/data'

In [17]:
# encoding=utf-8
"""
    Created on 10:38 2018/11/10 
    @author: Jindong Wang
"""

import numpy as np
from torch.utils.data import Dataset, DataLoader

In [15]:
# This is for parsing the X data, you can ignore it if you do not need preprocessing
def format_data_x(datafile):
    x_data = None
    for item in datafile:
        item_data = np.loadtxt(item, dtype=np.float)
        if x_data is None:
            x_data = np.zeros((len(item_data), 1))
            print(len(item_data))
        x_data = np.hstack((x_data, item_data))
    print("1", x_data.shape)   
    x_data = x_data[:, 1:]
    print("2",x_data.shape)
    X = None
    for i in range(len(x_data)):
        row = np.asarray(x_data[i, :])
        row = row.reshape(9, 128).T
        if X is None:
            X = np.zeros((len(x_data), 128, 9))
        X[i] = row
    print(X.shape)
    print("2xdata",X)
    return X


# This is for parsing the Y data, you can ignore it if you do not need preprocessing
def format_data_y(datafile):
    data = np.loadtxt(datafile, dtype=np.int) - 1
    YY = np.eye(6)[data]
    return YY


In [5]:
def onehot_to_label(y_onehot):
    a = np.argwhere(y_onehot == 1)
    return a[:, -1]


class data_loader(Dataset):
    def __init__(self, samples, labels, t):
        self.samples = samples
        self.labels = labels
        self.T = t

    def __getitem__(self, index):
        sample, target = self.samples[index], self.labels[index]
        if self.T:
            return self.T(sample), target
        else:
            return sample, target

    def __len__(self):
        return len(self.samples)


def normalize(x):
    x_min = x.min(axis=(0, 2, 3), keepdims=True)
    x_max = x.max(axis=(0, 2, 3), keepdims=True)
    x_norm = (x - x_min) / (x_max - x_min)
    return x_norm



In [11]:
# Load data function, if there exists parsed data file, then use it
# If not, parse the original dataset from scratch
def load_data(data_folder):
    import os
    if os.path.isfile(data_folder + 'data_har.npz') == True:
        data = np.load(data_folder + 'data_har.npz')
        X_train = data['X_train']
        Y_train = data['Y_train']
        X_test = data['X_test']
        Y_test = data['Y_test']
    else:
        # This for processing the dataset from scratch
        # After downloading the dataset, put it to somewhere that str_folder can find
        str_folder = data_folder
        INPUT_SIGNAL_TYPES = [
            "body_acc_x_",
            "body_acc_y_",
            "body_acc_z_",
            "body_gyro_x_",
            "body_gyro_y_",
            "body_gyro_z_",
            "total_acc_x_",
            "total_acc_y_",
            "total_acc_z_"
        ]

        str_train_files = [str_folder + 'train/' + 'Inertial Signals/' + item + 'train.txt' for item in
                           INPUT_SIGNAL_TYPES]
        str_test_files = [str_folder + 'test/' + 'Inertial Signals/' +
                          item + 'test.txt' for item in INPUT_SIGNAL_TYPES]
        str_train_y = str_folder + 'train/y_train.txt'
        str_test_y = str_folder + 'test/y_test.txt'

        print(str_train_files)
        X_train = format_data_x(str_train_files)
        # print(X_train)
        X_test = format_data_x(str_test_files)
        Y_train = format_data_y(str_train_y)
        Y_test = format_data_y(str_test_y)

    # return X_train, onehot_to_label(Y_train), X_test, onehot_to_label(Y_test)

In [16]:
load_data(path)

['/Users/lizliao/Downloads/GitFYP_experiment/Dataset/UCI HAR Dataset/data/train/Inertial Signals/body_acc_x_train.txt', '/Users/lizliao/Downloads/GitFYP_experiment/Dataset/UCI HAR Dataset/data/train/Inertial Signals/body_acc_y_train.txt', '/Users/lizliao/Downloads/GitFYP_experiment/Dataset/UCI HAR Dataset/data/train/Inertial Signals/body_acc_z_train.txt', '/Users/lizliao/Downloads/GitFYP_experiment/Dataset/UCI HAR Dataset/data/train/Inertial Signals/body_gyro_x_train.txt', '/Users/lizliao/Downloads/GitFYP_experiment/Dataset/UCI HAR Dataset/data/train/Inertial Signals/body_gyro_y_train.txt', '/Users/lizliao/Downloads/GitFYP_experiment/Dataset/UCI HAR Dataset/data/train/Inertial Signals/body_gyro_z_train.txt', '/Users/lizliao/Downloads/GitFYP_experiment/Dataset/UCI HAR Dataset/data/train/Inertial Signals/total_acc_x_train.txt', '/Users/lizliao/Downloads/GitFYP_experiment/Dataset/UCI HAR Dataset/data/train/Inertial Signals/total_acc_y_train.txt', '/Users/lizliao/Downloads/GitFYP_experimen

/var/folders/vp/5q7d1xdj6xn18sv64g2jrcdr0000gn/T/ipykernel_17706/654061952.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  item_data = np.loadtxt(item, dtype=np.float)


1 (7352, 1153)
2 (7352, 1152)
(7352, 128, 9)
2xdata [[[ 1.808515e-04  1.076681e-02  5.556068e-02 ...  1.012817e+00
   -1.232167e-01  1.029341e-01]
  [ 1.013856e-02  6.579480e-03  5.512483e-02 ...  1.022833e+00
   -1.268756e-01  1.056872e-01]
  [ 9.275574e-03  8.928878e-03  4.840473e-02 ...  1.022028e+00
   -1.240037e-01  1.021025e-01]
  ...
  [-1.147484e-03  1.714439e-04  2.647864e-03 ...  1.018445e+00
   -1.240696e-01  1.003852e-01]
  [-2.222655e-04  1.574181e-03  2.381057e-03 ...  1.019372e+00
   -1.227451e-01  9.987355e-02]
  [ 1.575500e-03  3.070189e-03 -2.269757e-03 ...  1.021171e+00
   -1.213260e-01  9.498741e-02]]

 [[ 1.093752e-03 -4.687588e-03 -2.685954e-02 ...  1.018851e+00
   -1.239760e-01  9.792958e-02]
  [ 4.550077e-03 -7.487894e-03 -2.509841e-02 ...  1.022380e+00
   -1.268078e-01  9.935086e-02]
  [ 2.879173e-03 -8.429991e-03 -2.597534e-02 ...  1.020781e+00
   -1.277862e-01  9.811381e-02]
  ...
  [-4.646144e-03 -5.479850e-03 -2.996671e-04 ...  1.014788e+00
   -1.290268e-01

/var/folders/vp/5q7d1xdj6xn18sv64g2jrcdr0000gn/T/ipykernel_17706/654061952.py:27: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.loadtxt(datafile, dtype=np.int) - 1


In [10]:
load_data(path)

# format_data_x():

/var/folders/vp/5q7d1xdj6xn18sv64g2jrcdr0000gn/T/ipykernel_17706/3058114708.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  item_data = np.loadtxt(item, dtype=np.float)


7352
(7352, 1152)
(7352, 128, 9)
[[[ 1.808515e-04  1.076681e-02  5.556068e-02 ...  1.012817e+00
   -1.232167e-01  1.029341e-01]
  [ 1.013856e-02  6.579480e-03  5.512483e-02 ...  1.022833e+00
   -1.268756e-01  1.056872e-01]
  [ 9.275574e-03  8.928878e-03  4.840473e-02 ...  1.022028e+00
   -1.240037e-01  1.021025e-01]
  ...
  [-1.147484e-03  1.714439e-04  2.647864e-03 ...  1.018445e+00
   -1.240696e-01  1.003852e-01]
  [-2.222655e-04  1.574181e-03  2.381057e-03 ...  1.019372e+00
   -1.227451e-01  9.987355e-02]
  [ 1.575500e-03  3.070189e-03 -2.269757e-03 ...  1.021171e+00
   -1.213260e-01  9.498741e-02]]

 [[ 1.093752e-03 -4.687588e-03 -2.685954e-02 ...  1.018851e+00
   -1.239760e-01  9.792958e-02]
  [ 4.550077e-03 -7.487894e-03 -2.509841e-02 ...  1.022380e+00
   -1.268078e-01  9.935086e-02]
  [ 2.879173e-03 -8.429991e-03 -2.597534e-02 ...  1.020781e+00
   -1.277862e-01  9.811381e-02]
  ...
  [-4.646144e-03 -5.479850e-03 -2.996671e-04 ...  1.014788e+00
   -1.290268e-01  9.353520e-02]
  [

/var/folders/vp/5q7d1xdj6xn18sv64g2jrcdr0000gn/T/ipykernel_17706/3058114708.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.loadtxt(datafile, dtype=np.int) - 1


In [7]:
def load(data_folder, batch_size=64):
    x_train, y_train, x_test, y_test = load_data(data_folder)
    x_train, x_test = x_train.reshape(
        (-1, 9, 1, 128)), x_test.reshape((-1, 9, 1, 128))
    transform = None
    train_set = data_loader(x_train, y_train, transform)
    test_set = data_loader(x_test, y_test, transform)
    train_loader = DataLoader(
        train_set, batch_size=batch_size, shuffle=True, drop_last=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

In [8]:
load_data(path)

/var/folders/vp/5q7d1xdj6xn18sv64g2jrcdr0000gn/T/ipykernel_17706/3058114708.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  item_data = np.loadtxt(item, dtype=np.float)


7352
(7352, 1152)
(7352, 128, 9)
2947


KeyboardInterrupt: 